# 잠재 디리클레 할당(Latent Dirichlet Allocation, LDA)

LSA : DTM을 차원 축소 하여 축소 차원에서 근접 단어들을 토픽으로 묶는다.  
LDA : 단어가 특정 토픽에 존재할 확률과 문서에 특정 토픽이 존재할 확률을 결합확률로 추정하여 토픽을 추출한다.

### LDA의 수행하기

1) 사용자는 알고리즘에게 토픽의 개수 k를 알려줍니다.  
2) 모든 단어를 k개 중 하나의 토픽에 할당합니다.  
3) 이제 모든 문서의 모든 단어에 대해서 아래의 사항을 반복 진행합니다. (iterative)  
3-1) 어떤 문서의 각 단어 w는 자신은 잘못된 토픽에 할당되어져 있지만, 다른 단어들은 전부 올바른 토픽에 할당되어져 있는 상태라고 가정합니다. 이에 따라 단어 w는 아래의 두 가지 기준에 따라서 토픽이 재할당됩니다.

#### 1) 정수 인코딩과 단어 집합 만들기

- 텍스트 전처리

In [1]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups

In [2]:
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data
len(documents)

11314

In [3]:
news_df = pd.DataFrame({'document':documents})
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w) > 3]))
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

In [4]:
from nltk.corpus import stopwords

In [5]:
stop_words = stopwords.words('english')
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split())
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

In [6]:
tokenized_doc[:5]

0    [well, sure, story, seem, biased, disagree, st...
1    [yeah, expect, people, read, actually, accept,...
2    [although, realize, principle, strongest, poin...
3    [notwithstanding, legitimate, fuss, proposal, ...
4    [well, change, scoring, playoff, pool, unfortu...
Name: clean_doc, dtype: object

In [7]:
from gensim import corpora

In [8]:
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[0])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 2), (22, 2), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 4), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 2), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1)]


In [9]:
print(dictionary[5])

blessing


In [10]:
len(dictionary)

64281

#### 2) LDA 모델 훈련시키기

In [11]:
import gensim

In [12]:
# 토픽 k = 20
NUM_TOPICS = 20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)
for topics in topics:
    print(topics)

(0, '0.014*"like" + 0.012*"would" + 0.011*"know" + 0.010*"time"')
(1, '0.008*"borland" + 0.007*"typing" + 0.007*"rockefeller" + 0.006*"layout"')
(2, '0.012*"people" + 0.010*"would" + 0.007*"believe" + 0.007*"think"')
(3, '0.012*"windows" + 0.009*"thanks" + 0.009*"drive" + 0.008*"system"')
(4, '0.066*"window" + 0.039*"widget" + 0.016*"application" + 0.014*"xlib"')
(5, '0.027*"water" + 0.014*"pitcher" + 0.009*"cooling" + 0.008*"plants"')
(6, '0.027*"armenian" + 0.020*"armenians" + 0.019*"turkish" + 0.012*"turkey"')
(7, '0.031*"period" + 0.018*"power" + 0.011*"scorer" + 0.010*"play"')
(8, '0.006*"cause" + 0.005*"medical" + 0.005*"pain" + 0.005*"disease"')
(9, '0.008*"president" + 0.006*"national" + 0.005*"april" + 0.005*"year"')
(10, '0.024*"space" + 0.015*"output" + 0.015*"file" + 0.014*"entry"')
(11, '0.016*"people" + 0.012*"said" + 0.009*"children" + 0.009*"would"')
(12, '0.022*"price" + 0.021*"sale" + 0.017*"shipping" + 0.017*"offer"')
(13, '0.019*"available" + 0.013*"information" + 0

#### 3) LDA 시각화 하기

In [13]:
import pyLDAvis.gensim

In [14]:
pyLDAvis.enable_notebook()

In [15]:
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

C:\Users\it\.conda\envs\protf\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [16]:
pyLDAvis.display(vis)

#### 4) 문서 별 토픽 분포 보기

In [18]:
for i, topic_list in enumerate(ldamodel[corpus]):
    if i==5:
        break
    print(i,'번째 문서의 topic 비율은',topic_list)

0 번째 문서의 topic 비율은 [(0, 0.15203173), (2, 0.13071623), (6, 0.12389246), (8, 0.24934964), (9, 0.23338842), (18, 0.09931633)]
1 번째 문서의 topic 비율은 [(0, 0.23496902), (2, 0.53187835), (5, 0.028774451), (15, 0.18331257)]
2 번째 문서의 topic 비율은 [(9, 0.10951156), (14, 0.67356235), (18, 0.20318809)]
3 번째 문서의 topic 비율은 [(8, 0.17518176), (13, 0.019247908), (14, 0.7236812), (16, 0.014940488), (19, 0.055195373)]
4 번째 문서의 topic 비율은 [(0, 0.20773168), (10, 0.06677469), (15, 0.6940031)]


In [23]:
def make_topictable_per_doc(ldamodel, corpus, texts):
    topic_table = pd.DataFrame()
    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        
        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc):
            #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j==0: # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
            else:
                break
    return(topic_table)

In [24]:
topictable = make_topictable_per_doc(ldamodel, corpus, tokenized_doc)
topictable = topictable.reset_index()
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:10]

,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,8.0,0.2494,"[(0, 0.15156998), (2, 0.1311821), (6, 0.123962..."
1,1,2.0,0.5319,"[(0, 0.23494157), (2, 0.5318992), (5, 0.028774..."
2,2,14.0,0.6736,"[(9, 0.10951321), (14, 0.67356086), (18, 0.203..."
3,3,14.0,0.7237,"[(8, 0.17517513), (13, 0.019247405), (14, 0.72..."
4,4,15.0,0.6939,"[(0, 0.20786996), (10, 0.06671549), (15, 0.693..."
5,5,0.0,0.4445,"[(0, 0.44453132), (1, 0.045649692), (3, 0.1275..."
6,6,3.0,0.3632,"[(0, 0.23108931), (3, 0.36324763), (4, 0.01535..."
7,7,0.0,0.3100,"[(0, 0.30998775), (2, 0.2662358), (7, 0.086224..."
8,8,0.0,0.4974,"[(0, 0.49743587), (9, 0.24410243), (10, 0.0861..."
9,9,14.0,0.7661,"[(3, 0.04858958), (14, 0.7660645), (19, 0.1733..."
